# Chapter 6: Mining Frequent Patterns, Associations & Correlations

## Frequent Pattern Analysis

- **Frequent patterns** in a data set
    - *a set of items*
    - *subsequences*
    - *substructures*
- Examples
    - Web log
    - Road traffic

## Basic Concepts

- **Frequent itemset**
    - $X = \{x_1, x_2, \ldots, x_k\}$
- **Association rule** $X \Rightarrow Y$
    - **support**: probability that a transaction contains $X \cup Y$
    - **confidence**: conditional probability that a transaction containing $X$ also contains $Y$
    - **minimum support, minimum confidence**

## Example

|Tid|Items|
|---|---|
|1|A,B,D|
|2|A,C,D|
|3|A,D,E|
|4|B,E,F|
|5|B,C,D,E,F|

- Let min_sup = 50%, min_conf = 50%
- Frequent patterns
    - A **3**, B **3**, D **4**, E **3**, AD **3**
- Association rules
    - A $\Rightarrow$ D (**60**%, **100**%)
    - D $\Rightarrow$ A (**60**%, **75**%)

## Mining Association Rules

- Two-step process
    - find all **frequent itemsets** (w/ min_sup)
    - generate **strong association rules** from the frequent itemsets (min_sup, min_conf)
- A long pattern contains a combinatorial **number of subpatterns** (e.g., 100 items)
    - $\displaystyle {100 \choose 1} + {100 \choose 2} + \cdots + {100 \choose 100} = 2^{100} - 1 \approx 1.27 \times 10^{30}$

## Closed & Max Patterns

- Solution: mine closed patterns & max-patterns
- **Closed pattern** $X$
    - no super-pattern $Y \supset X$ w/ the same support
- **Max-pattern** $X$
    - no super-pattern $Y \supset X$
- Closed pattern is a lossless compression of frequent patterns
    - reducing the number of patterns and rules

## Example

- $\{<a_1,\ldots,a_{100}>,<a_1,\ldots,a_{50}>\}$, min_sup = 0.5
- Frequent pattern?
    - all item combinations
- Closed pattern?
    - $<a_1,\ldots,a_{100}>$: 1
    - $<a_1,\ldots,a_{50}>$: 2
- Max-pattern?
    - $<a_1,\ldots,a_{100}>$: 1

## Apriori Algorithm

- **Apriori property**
    - subset of a freq. itemset is also frequent
    - e.g., {beer, diaper, nuts}, {beer, diaper}
- **Apriori pruning**
    - if X is infrequent,
    - then superset of X is pruned
- Procedure
    1. scan DB to get freq. **1-itemset**
    1. generate candidate (k+1)-itemsets from freq. **k-itemsets**
    1. test candidate **(k+1)-itemsets** against DB
    1. stop when no freq. or candidate itemsets can be generated

![Apriori Algorithm Example](./img/6.1.png)

## Important Details

- **Self-joining** of k-itemsets to generate (k+1)-itemsets
    - two k-itemsets are joined if their first (k-1) items are the same
- **Pruning**: remove if subset not frequent
- Example: L3 = {abc, abd, acd, ace, bcd}
    - abc and abd $\Rightarrow$ abcd
    - acd and ace $\Rightarrow$ acde
    - acde pruned because ade is not in L3

## Interestingness Measure

- Association rule
    - A $\Rightarrow$ B [support, confidence]
- A strong association rule
    - play basketball $\Rightarrow$ eat cereal [40%, 66.7%]
- The rule is misleading
    - overall, 75% of students eat cereal
    - play basketball $\Rightarrow$ not eat cereal [20%, 33.3%]

## Correlation Rules

- Correlation rule
    - A $\Rightarrow$ B [support, confidence, **correlation**]
- Measure of dependent/correlated events
    - $\displaystyle lift(A, b) = \frac{P(A \cup B)}{P(A)P(B)}$
- lift = 1? $\quad$ **independent**
- lift < 1? $\quad$ **negatively dependent**
- lift > 1? $\quad$ **positively dependent**

![Correlation Rules](./img/6.2.png)

## Frequent Pattern Mining

- Challenges
    - multiple **scans** of the whole data set
    - a huge number of **candidates**
    - tedious **support counting** for candidates
- Improving Apriori: general ideas
    - reduce data scans
    - reduce number of candidates
    - facilitate support counting of candidates

## Partition: Two Data Scans

- A frequent itemset must be frequent in at least one partition
- Partition size? # of partitions?
    - each partition fits into main memory

![Partition Two Data Scans](./img/6.3.png)

## Sampling for Freq. Patterns

- Select a sample data set
- Mine frequent patterns within sample
    - may use a lower min_sup
- Scan whole data set for actual support
    - only check closed patterns
    - e.g., check **abcd** instead of **ab**, **acd**, ..., etc.
- Scan again to find missed frequent patterns
- Sample size?

## Transaction Reduction

- If a transaction T does not contain any frequent k-itemset
    - then for any h > k, no need to check T when searching for frequent h-itemset
- Implementation
    - sequential scan
    - vs. random access

## Reduce #Candidates

- Hash itemsets to buckets
- If a hash bucket count is below support threshold
    - them itemsets in that hash bucket are not frequent itemsets

## Dynamic Itemset Counting

- If A & D are freq., start count for AD
- If BC, BD, CD are freq., start count for BCD

## Count Support of Candidates

- Why counting candidate support a problem?
    - #candidates: total, per transaction
- Method
    - store candidate itemsets in a **hash-tree**
    - **leaf-node** contains a list of itemsets and counts
    - **interior node** contains a hash table
    - **subset function**: finds all candidates contained in a transaction

![Count Support of Candidates](./img/6.4.png)

## Vertical Data Format

- **Horizontal** data format
    - T1: {A, D, E, F}
- **Vertical** data format
    - t(AD) = {T1, T6, ...}
- Derive closed pattern via **vertical intersection**
    - t(X) = {T1, T2, T3} and t(Y) = {T1, T3, T4}
    - t(XY) = {T1, T3}

## Frequent Itemset Mining

- Multiple **data scans** are costly
- Mining **long patterns** needs many scans and generates lots of candidates
    - e.g., 100 itemsets: #scans, $candidates
- **Bottleneck**: candidate generation & test
- Can we avoid candidate generation?

## FP-growth

- Find frequent itemsets without candidate generation
- Grow long patterns from short ones using local frequent items
- Example
    - **abc** is a frequent itemset; get all transacitons with abc: **DB | abc**
    - **d** is a local frequent item in DB | abc
    - then **abcd** is a frequent itemset
- Idea: Frequent pattern growth
    - recursively grow freq. patterns by pattern and data partition
- Method
    - freq. item $\Rightarrow$ conditional pattern base $\Rightarrow$ conditional FP-tree
    - repeat on each newly created FP-tree
    - until FP-tree is empty or single path

## FP-tree Construction

- Scan, find freq. 1-itemset
- Sort freq. items in descending frequency
- Scan, construct FP-tree

## Conditional Pattern Base

- Traverse links of each frequent item, prefix paths

![Conditional Pattern Base](./img/6.5.png)

## Conditional FP-trees

![Conditional FP trees](./img/6.6.png)

## FP-growth vs. Apriori

![FP growth vs Apriori](./img/6.7.png)

## Other Correlation Measures

$\displaystyle all\_conf(A,B) = \frac{\sup(A \cup B)}{\max\{\sup(A),\sup(B)\}} = \min\{P(A|B),P(B|A)\}$

$max\_conf(A,B) = \max\{P(A|B),P(B|A)\}$

$\displaystyle Kulc(A,B) = \frac{1}{2}(P(A|B) + P(B|A))$

$\displaystyle cosine(A,B) = \frac{P(A \cup B)}{\sqrt{P(A) \times P(B)}} = \frac{\sup(A \cup B)}{\sqrt{\sup(A) \times \sup(B)}} = \sqrt{P(A|B) \times P(B|A)}$

## Comparison

![Correlation measures comparison](./img/6.8.png)

- **Null-transaction**: e.g., $\neg m, \neg c$
- **Null-variant**: lift and $X^2$
- **Null-invariant**: all_conf, max_conf, Kulc, cosine
- **Imbalance ratio**
    - $\displaystyle IR(A,B) = \frac{|\sup(A) - \sup(B)|}{\sup(A) + \sup(B) - \sup(A \cup B)}$